**To use this notebook:** Run one line at a time waiting for each cell to return before running the next cell.

# Icons

Icons can be applied to both the `Title` of a `Panel` [widgets](./widgets.ipynb) and [commands](./commands.ipynb), providing more customization than `icon_class`.

In [ ]:
%pip install -q ipylab

In [ ]:
import json

import ipywidgets as ipw
import traitlets

import ipylab

## SVG

An icon requires both a _unique_ name, as well as an SVG string. There are some [guidelines](https://jupyterlab.readthedocs.io/en/stable/extension/ui_components.html#labicon-set-up-and-render-icons) for creating "good" icons. For example:
- don't include the `<?xml>` declaration
- don't use `ids`
- don't specify a `width` or `height`
  - ensures the icon can be used in a number of settings
- use the `jp-icon*` classes on filled items
  - ensures the icon looks good on light and dark themes

In [ ]:
SVG = """<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24">
  <circle class="jp-icon-selectable jp-icon3" cx="12" cy="12" r="12" fill="#616161" />
  <path class="jp-contrast0" fill="#fff" d="M9 16.17L4.83 12l-1.42 1.41L9 19 21 7l-1.41-1.41z"/>
</svg>"""

Icons can be displayed directly, and sized with the `layout` member inherited from `ipywidgets.DOMWidget`.

In [ ]:
icon = ipylab.Icon(name="my-icon", svgstr=SVG, layout={"width": "32px"})
icon

### More about `jp-icon` classes
The interactive below isn't particularly _robust_, but shows how the different `jp-icon-*` classes can be used.

In [ ]:
icon_prefix = ["", "-accent", "-brand", "-contrast", "-warn"]
options = [""] + [f"jp-icon{sub}{i}" for sub in icon_prefix for i in range(5)]
background = ipw.SelectionSlider(description="background", options=options)
foreground = ipw.SelectionSlider(description="foreground", options=options)

traitlets.dlink((background, "value"), (icon, "svgstr"), lambda x: SVG.replace("jp-icon3", x))
traitlets.dlink((foreground, "value"), (icon, "svgstr"), lambda x: SVG.replace("jp-contrast0", x))
size = ipw.FloatSlider(32, description="size")
traitlets.dlink((size, "value"), (icon.layout, "width"), "{}px".format)
icon_controls = ipw.VBox([background, foreground, size, icon])
icon_controls

## Icons on Panel Titles

Once defined, an icon can be used on a panel title in place of `icon_class` 

In [ ]:
panel = ipylab.Panel([icon_controls])
panel.title.icon = icon
traitlets.dlink((background, "value"), (panel.title, "label"))
panel.add_to_shell(mode=ipylab.InsertMode.split_right)

### More Title Options

Titles can also include a number of other options.

In [ ]:
def noop(value):
    return value


def as_json(value):
    try:
        return json.loads(value)
    except Exception:
        return {}


title_controls = []
for field_name in ["label", "caption", "icon_class", "class_name", "dataset"]:
    link_fn = noop
    placeholder = ""
    if field_name == "dataset":
        placeholder = "{}"
        link_fn = as_json
    field = ipw.Text(description=field_name, placeholder=placeholder)
    traitlets.dlink((field, "value"), (panel.title, field_name), link_fn)
    title_controls.append(field)
panel.children = [icon_controls, *title_controls]

## Icons on Commands

Icons can also assigned to [commands](./commands.ipynb) to provide additional context. 

In [ ]:
import asyncio
import random


async def randomize_icon(count=10):
    for _ in range(count):
        background.value = random.choice(options)  # noqa: S311
        await asyncio.sleep(0.1)

In [ ]:
t = ipylab.app.commands.add_command(
    "randomize",
    randomize_icon,
    label="Randomize My Icon",
    icon=icon,
)

In [ ]:
cmd = t.result()

We can use methods on `cmd` (Connection for the cmd registered in the Frontend) to add it to the command pallet, and create a launcher.

In [ ]:
t = cmd.add_to_command_pallet("All My Commands", rank=1000)

In [ ]:
cmd.add_launcher("my launcher")

In [ ]:
assert cmd in ipylab.app.commands.connections  # noqa: S101

Then open the _Command Palette_ (keyboard shortcut is `CTRL + SHIFT + C`).

The and _Launcher_ is `CTRL + SHIFT + L`. 

In [ ]:
asyncio.get_running_loop().call_later(1, ipylab.app.commands.execute, "apputils:activate-command-palette")

And run 'Randomize my icon'

In [ ]:
ipylab.app.commands.execute(cmd, {"count": 1})